In [54]:
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.metrics import mean_absolute_error


data = pd.read_csv("../input/train.csv")
data.dropna(axis = 0, subset = ["SalePrice"], inplace = True)
y = data.SalePrice
x = data.drop(["Id", "SalePrice"], axis = 1)
x = pd.get_dummies(x)
#trainX, testX, trainY, testY = train_test_split(x.as_matrix(), y.as_matrix(), test_size=0.25)

myImputer = Imputer()
trainX = pd.DataFrame(myImputer.fit_transform(x))
trainX.columns = x.columns
print(trainX.columns)
#testX = myImputer.transform(testX)
trainY = y



myModel = XGBRegressor(n_estimators = 1500, learning_rate = 0.02)
myModel.fit(trainX, trainY, verbose = False)

preds = myModel.predict(trainX)

scores = cross_val_score(myModel, trainX, trainY, cv = 5, scoring='neg_mean_absolute_error')
print(scores)
print('Mean Absolute Error %2f' %(-1 * scores.mean()))


###########################################
subm = pd.read_csv('../input/test.csv')
subm = pd.get_dummies(subm)


submX =  pd.DataFrame(myImputer.fit_transform(subm))
submX.columns = subm.columns

final_train, final_test = trainX.align(submX, join='left', axis=1)
predPrices = myModel.predict(final_test)


my_submission = pd.DataFrame({'Id': subm.Id, 'SalePrice': predPrices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

